## Camadas

Uma das maneiras mais comuns de combinar gráficos é utilizando a função _layer_. Caso os gráficos tenham domínios compatíveis podemos juntá-los para formas _eixos compartilhados_. Caso os codificadores de `x` ou `y` não forem compatíveis teremos que criar um _gráfico de eixo duplo_ que se sobrepõe utilizando escalas e eixos diferentes.

### Eixos compartilhados


Comecemos exibindo a temperátura média máxima e mínima por mês:

In [ ]:
alt.Chart(weather).mark_area().encode(
  alt.X('month(date):T', title="Mês"),
  alt.Y('average(temp_max):Q', title='Temperatura média máxima, Temperatura média mínima'),
  alt.Y2('average(temp_min):Q')
)

_O gráfico nos mostra o intervalo de temperaturas médias a cada mês considerando todos os dados. No entanto, isso não nos diz muito já que está misturando os dados de Nova Yotk e Seattle._

Vmoa então dividir os dados baseados nas cidades por cores, mudando também a opcaidade para considerar áreas de sobreposição.

In [ ]:
alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title="Mês"),
  alt.Y('average(temp_max):Q', title='Temperatura média máxima, Temperatura média mínima'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N', title='Cidade')
)

_Podemos perceber que Seattle é mais temperado, tendo temperaturas mais amenas durante o verão e mais quente durante o inverno_

nesse caso, criamos um gráfico em camadas sem características mais complexas simplesmente dividindo as áreas por cor. Já que o gráfico acima nos mostra os interalos das temperaturas também é válido mostrar o meio do intervalo.

Vamos criar um gráfico mostrando o ponto médio da temperatura média. Usaremos a transformação `Calculate` para calcular os pontos médios entre a temperatura média mínima e máxima.

In [ ]:
alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T', title='Mês'),
  alt.Y('average(temp_mid):Q', title= 'Temperatura do ponto médio'),
  alt.Color('location:N', title='Cidade')
)

Observação: Note o uso do `+datum.temp_min` na tranformação acima. Já que estamos carregando os dados direto de um arquivo CSV sem nenhuma especificação os valores de temperatura podem estar sendo representados internamente como strings. Adicionando `+` na frente dos valores os força a serem lidos como números.

Agora nós gostaríaoms de combinar os gráficos colocando os pontos médios por cima das áreas das temperaturas. Utilizando a sintax `chart1 + chart2`, nós podemos especificar um novo gráfico combinado em que `chart1` é a primeira camada e `chart2` é a segunda camada no topo:

In [ ]:
tempMinMax = alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title='Mês'),
  alt.Y('average(temp_max):Q', title='Temperatura média máxima, Temperatura média mínima'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T', title='Mês'),
  alt.Y('average(temp_mid):Q', title='Temperatura do ponto médio'),
  alt.Color('location:N', title= 'Cidade')
)

tempMinMax + tempMid

_Agora temos um gráfico de várias camadas! Contido o título do eixo-y ficou meio confuso..._

Vamos personalizar os eixos para deixar nosso gráfico mais limpo. Se definirmos um título para uma camada isso valerá para todas



In [ ]:
tempMinMax = alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T', title='Mês'),
  alt.Y('average(temp_mid):Q', title= 'Temperatura média °C'),
  alt.Color('location:N', title= 'Cidade')
)

tempMinMax + tempMid

_O que aconteceria se ambas camadas tivessem eixos personalizados? Modifique o código acima para descobrir_

Utilizamos acima `+` que é um atalho para o método `layer`. Podemos criar um gráfico idêntico usando o método `layer` diretamente:

In [ ]:
alt.layer(tempMinMax, tempMid)

Perceba que a ordem das camadas é relevante para qual vai sobrepor qual. _Exeprimente mudar a ordem e ver os resultados. O que acontece?(Dica: preste atenção na cor da linha)_

### Gráfico com eixos duplos

_Seattle tem fama de chuvosa, isso é merecido?_

Vamos olhar para a precipitação junto com a temperatura para analisar melhor. Vamos primeiramente montar um gráfico base para mostrar a precipitação em Seattle:

In [ ]:
alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T', title=None),
  alt.Y('average(precipitation):Q', title='Precipitação')
)

Para facilitar a comparação com os dados da temperatura vamos criar um gráfico de camadas. Isso é o que acontece se tentarmos sobrepor os gráficos como fizemos antes:

In [ ]:
tempMinMax = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura média °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitação')
)

alt.layer(tempMinMax, precip)

_Os valores de precipitação são consideravelmente menores que os de temperatura, ou seja, usam um intervalo do eixo-y bem menor!_

Por padrâo, os gráficos em camadas usam um _domínio cmoporatilhado_: os valores dos eixos x e y são combinados por todas as camadas para obter uma extensão compratilhada adequada. Esse comportamento padrão assume que as camadas compratilham as mesmas unidades nos eixos. Entretanto, isso não se aplica a esse exemplo, já que estamos unindo valores de tempreatura (graus Celsius) e precipitação (polegadas)!

Se quisermos usar escalas diferentes no eixo y, precisamos especificar para o Altair que queremos diferenciar os dados por camada. Nesse caso, queremos que as escalas e domínios do eixo y sejam independentes e não compartilhados. O objeto `Chart` inclui o método `resolved_scale` que lida com isso deixando-nos escolher o que desejamos:

In [ ]:
tempMinMax = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura média °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitação')
)

alt.layer(tempMinMax, precip).resolve_scale(y='independent')

Agora podemos perceber que o Outono é a estação mais chuvosa em Seattle (tendo seu pico em novembro), seguido de verôes mais secos.

Você pode ter notado um redundância nas especificações dos nossos gráficos: ambos usam a mesma base de dados e tem um filtro para usar apenas dados de Seattle. Se quiser pode simplificar o código especificando a base de dados e o filtro para o gráfico de maior nível das camadas. As outras camadas vão herdar os dados se não for especificado nada diferente:

In [ ]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura média °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart().mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitação')
)

alt.layer(tempMinMax, precip, data=weather).transform_filter(
  'datum.location == "Seattle"'
).resolve_scale(y='independent')

Por mais que exios duplos sejam uteis eles são _propensos a interpretações precipitadas_, já que diferentes unidades e escalas no mesmo eixo podem ser confuso. Já que é viável, é bom considerar transformações que mapeiam diferentes campos de dados em uma mesma unidade, poe exemplo mostrando  [quantiles](https://en.wikipedia.org/wiki/Quantile) ou mudança percentual relativa. 